In [1]:
# !pip install skicit-learn

import sklearn
import pandas as pd
import json
import numpy as np

In [2]:
with open('new_new_fights_df.json', 'r') as file:
    fights = json.load(file)
    df = pd.DataFrame(fights)

with open('fight_results.json', 'r') as file:
    y = json.load(file)

print(len(df), len(df.columns))
print(len(y))
print(df.head())

1219 150
1219
   Fight_info_division  Fight_info_n_rounds  Fighter_A_basic_stats_Height  \
0                    2                    5                            71   
1                    5                    3                            72   
2                    2                    3                            70   
3                    5                    5                            75   
4                    3                    3                            72   

   Fighter_A_basic_stats_Reach  Fighter_A_basic_stats_Stance_0  \
0                           70                               1   
1                           73                               0   
2                           75                               1   
3                           74                               0   
4                           74                               0   

   Fighter_A_basic_stats_Stance_1  Fighter_A_basic_stats_Age  \
0                               1                      33.03  

In [4]:
import numpy as np
from feature_engineering import engineer_features

new_df = engineer_features(df)
with open('X.json', "w") as file:
    json.dump(new_df.to_dict(orient="records"), file)

with open('X.json', "r") as file:
    X = json.load(file)
    X = pd.DataFrame.from_dict(X)

print(X.head())

   Fight_kd_diff  Fight_striking_ovr_acc_diff  Fight_mean_striking_diff  \
0      -0.163561                    -0.037561                  0.789150   
1       0.049969                    -0.007785                 -0.697136   
2       0.393696                    -0.071537                 -0.559586   
3      -1.053821                    -0.023387                  0.834854   
4      -0.011906                    -0.012827                 -0.623442   

   Fight_strikes_head_diff  Fight_strikes_body_diff  Fight_strikes_leg_diff  \
0                -0.124798                -0.040354               -0.143449   
1                 0.657837                -0.260915               -0.119290   
2                 0.752665                 0.163419               -0.543225   
3                 1.138134                 0.168254               -1.388993   
4                -0.806657                -0.286590                0.208629   

   Fight_strikes_distance_diff  Fight_strikes_clinch_diff  \
0            

In [5]:
with open('fight_results.json', 'r') as file:
    y = json.load(file)
    y_df = pd.DataFrame.from_dict(y)

y_df['winner'] = y_df['winner'].replace({'A': 0, 'B': 1, 'Draw/NC': 2})
print(y_df.head())

                 id  winner
0  9b3bade95e1dd484       0
1  191ee545160a4e09       1
2  5ce29056e0cd2df9       0
3  633f4b5ec767769d       0
4  4072abf0b0c179f2       1


C:\Users\Rober\AppData\Local\Temp\ipykernel_23768\2140858880.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_df['winner'] = y_df['winner'].replace({'A': 0, 'B': 1, 'Draw/NC': 2})


In [6]:
import numpy as np

y = np.ravel(y_df['winner']) 

# get indices where y == 2 (draw/no contest)
# very rare, difficult to predict
indices_to_remove = np.where(y == 2)[0] # from test too?

X_filtered = np.delete(X, indices_to_remove, axis=0)
y_filtered = np.delete(y, indices_to_remove, axis=0)

print(X_filtered.shape, y_filtered.shape)

(1198, 70) (1198,)


**BASE/NAIVE MODEL: PREDICTING ON THE FAVOURITE. PROBLEM => ACCESSING ODDS**

In [17]:
train_set_size = int(len(X_filtered)*0.6)
val_set_size = int(len(y_filtered)*0.2)
# X = X_std[:,:12]
X_train = X_filtered[-train_set_size:]
X_val = X_filtered[val_set_size:-train_set_size]
X_test = X_filtered[:-train_set_size-val_set_size]
y_train = y_filtered[-train_set_size:]
y_val = y_filtered[val_set_size:-train_set_size]
y_test = y_filtered[:-train_set_size-val_set_size]

print(X_filtered.shape, y_filtered.shape)

print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)

(1198, 70) (1198,)
(718, 70) (718,) (241, 70) (241,) (241, 70) (241,)


**ML models**

**Check train results**

In [18]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_std = scaler.fit_transform(X_train)
X_val_std = scaler.transform(X_val)

kernel = DotProduct() + WhiteKernel()
gpr = GaussianProcessRegressor(kernel=kernel, random_state=0).fit(X_train_std, y_train)
print(gpr.score(X_train_std, y_train))
print(f"Train score for Gaussian: {gpr.score(X_val_std, y_val)}") # bad validation performance?

0.16978394460020363
-0.008492554508390704


In [19]:
# !pip install xgboost

import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_val_std = scaler.transform(X_val)


best_r2 = 0
best_config = None

# xgb_model = xgb.XGBRegressor()
# xgb_model.load_model("xgb_best_model.json") # the problem is that i should only load the parameters actually, not the fitted model
xgb_model = xgb.XGBRegressor(
    n_estimators=500,
    max_depth=11,
    learning_rate=0.003,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

xgb_model.fit(X_train_std, y_train)

# train evaluation
y_train_pred = xgb_model.predict(X_train_std)
train_mse = mean_squared_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)
print(f"Train MSE: {train_mse:.4f}, Train R2: {train_r2:.4f}")

# validation
y_val_pred = xgb_model.predict(X_val_std)
y_val_pred_adj = np.round(y_val_pred).astype(int)
acc = len(np.where(y_val_pred_adj == y_val)[0]) / len(y_val_pred_adj)
test_mse = mean_squared_error(y_val, y_val_pred)
test_r2 = r2_score(y_val, y_val_pred)
print(f"Test MSE: {test_mse:.4f}, Val R2: {test_r2:.4f}\n") # bad generalization?

Train MSE: 0.0352, R2: 0.8572
Test MSE: 0.2413, R2: 0.0139



**Train + val: check test results**

In [20]:
X_trainval = np.concat((X_train, X_val))
y_trainval = np.concat((y_train, y_val))
scaler = StandardScaler()
X_trainval_std = scaler.fit_transform(X_trainval)
X_test_std = scaler.transform(X_test)

xgb_model = xgb.XGBRegressor(
    n_estimators=500,
    max_depth=11,
    learning_rate=0.003,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)
xgb_model.fit(X_trainval_std, y_trainval) # now train on whole dataset to predict hypothetical fights
y_test_pred = xgb_model.predict(X_test_std)
print(f"y_test_pred shape {y_test_pred.shape}")
xgb_r2_test = r2_score(y_test, y_test_pred)
print(f"Test R2 for XGBoost {test_r2}")
# wow so much better than validation?

kernel = DotProduct() + WhiteKernel()
gpr = GaussianProcessRegressor(kernel=kernel, random_state=0).fit(X_trainval_std, y_trainval)
gpr_r2_test = gpr.score(X_std, y_filtered)
print(f"Test score for Gaussian: {gpr_r2_test}") # bad generalization?

y_test_pred shape (241,)
Test R2 for XGBoost 0.11429893970489502
Score for Gaussian: 0.0832859957690566


In [ ]:
# check test results

row = y_df.loc[y_df['id'] == "d95f21817a56eb05"]
print(row.index[0])
index = row.index[0]# - 240

# print(gpr.predict(h_df.iloc[0].values.reshape(1,-1), return_std=True)[0])
# print(xgb_loaded.predict(h_df.iloc[0].values.reshape(1,-1)))

# print(gpr.predict(X_val_std[row.index[0]].reshape(1,-1), return_std=True)[1])

print(f"Gaussian: {round(float(gpr.predict(X_test_std[index].reshape(1,-1))[0]), 2)}")
print(f"XGBoost: {round(float(y_test_pred[index]), 2)}")

# print(h_df.iloc[0].to_numpy().reshape(1,-1))

# very bad prediction for chandla vs back at the flouty. To count => More age, damage accumulated and recovery stats
# pereira hill => bad prediction. To count =>  very long layoffs
# o malley merab predicts o malley clearly??. To count => More wrestling / cardio stats
# reyes krylov predicts krylov. To count => very long layoffs
# predicts Bryce to confidently beat Silva. Reward more the recent performances
# penalyze fighters who recently got knocked out like BSD, specially against strong punchers
# craig favourite against borralho??
# kattar ON A 2 FIGHT LOSING STREAK favourite against sterling??
# vera favourite against o mallley? why?
# ancient dos anjos favourite against gamrot. old record counting too much
# predicts bad for jones

**Train on all data to predict future fights**

In [22]:
# X_all = np.concat((X_train, X_val))
# y_trainval = np.concat((y_train, y_val))
full_data_scaler = StandardScaler()
X_std = full_data_scaler.fit_transform(X_filtered)
# X_test_std = scaler.transform(X_test)

xgb_model = xgb.XGBRegressor(
    n_estimators=500,
    max_depth=11,
    learning_rate=0.003,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)
xgb_model.fit(X_std, y_filtered) # now train on whole dataset to predict hypothetical fights
y_pred = xgb_model.predict(X_std)
print(f"y_pred shape {y_pred.shape}")
train_r2 = r2_score(y_filtered, y_pred)
print(f"All train R2 for XGBoost {test_r2}")
# exactly the same as for test? somethings off

kernel = DotProduct() + WhiteKernel()
gpr = GaussianProcessRegressor(kernel=kernel, random_state=0).fit(X_std, y_filtered)
print(f"Train score for Gaussian: {gpr.score(X_std, y_filtered)}")

y_pred shape (1198,)
All train R2 for XGBoost 0.11429893970489502
Train score for Gaussian: 0.14034525972787404


In [35]:
from datetime import datetime
from fight import get_hypothetical_fight
from fighter import get_fighter_name_from_url
from fight import encode_weight_class

def predict_hypothetical_fight(fighter_0_id, fighter_1_id, full_df, division, cutoff_date=None): # only works for future fights i think, doesnt filter by date
    url_0 = "http://ufcstats.com/fighter-details/"+fighter_0_id
    url_1 = "http://ufcstats.com/fighter-details/"+fighter_1_id

    fighter_0_name = get_fighter_name_from_url(url_0)
    fighter_1_name = get_fighter_name_from_url(url_1)

    if cutoff_date is None:
        cutoff_date = datetime.strptime("2025-09-20", "%Y-%m-%d").date() ## is age being calculated properly?
    
    hypothetical_fight = get_hypothetical_fight(url_0, url_1, cutoff_date)
    hypothetical_fight['Fighter_A_career_stats_weight'] = hypothetical_fight['Fighter_B_career_stats_weight'] = None
    # hypothetical_fight['Fight_info_division'] = np.array([division] * len(hypothetical_fight)).reshape(-1,1)
    hypothetical_fight['Fight_info_division'] = division
    h_df = engineer_features(hypothetical_fight, full_df)

    h_df = full_data_scaler.transform(h_df) # which scaler is this one?


    gaussian_prob = gpr.predict(h_df[0].reshape(1,-1), return_std=True)[0][0]
    xgb_prob = xgb_model.predict(h_df[0].reshape(1,-1))[0]

    gaussian_weight = gpr_r2_test / (gpr_r2_test + xgb_r2_test)
    xgb_weight = xgb_r2_test / (gpr_r2_test + xgb_r2_test)

    combined_prob = round((gaussian_prob * gaussian_weight + xgb_prob * xgb_weight) * 100, 2)

    print(f"Gaussian: {gpr.predict(h_df[0].reshape(1,-1), return_std=True)[0][0]}")
    print(f"XG Boost: {xgb_model.predict(h_df[0].reshape(1,-1))[0]}")
    
    # print((1-gpr.predict(h_df.iloc[0].values.reshape(1,-1), return_std=True)[0][0]) * 100 + "%")
    # print((1-xgb_loaded.predict(h_df.iloc[0].values.reshape(1,-1))[0]) * 100 + "%")

    return combined_prob

In [38]:
def predict_hypothetical_probabilities(fighter_a_id, fighter_b_id, division, date):
    combined_prob_1 = predict_hypothetical_fight(fighter_a_id, fighter_b_id, df, division, date)
    combined_prob_2 = predict_hypothetical_fight(fighter_b_id, fighter_a_id, df, division, date)
    real_prob = combined_prob_2 / (combined_prob_1+combined_prob_2)
    fighter_a_url, fighter_b_url = "http://ufcstats.com/fighter-details/" + fighter_a_id, "http://ufcstats.com/fighter-details/" + fighter_b_id
    fighter_a_name, fighger_b_name = get_fighter_name_from_url(fighter_a_url), get_fighter_name_from_url(fighter_b_url)
    print(f"Probabilities given for {fighter_a_name} to beat {fighter_b_name} in LW: {round(real_prob*100, 1)}%")

In [40]:
fighter_a_name = input("Please introduce fighter A name")
fighter_a_surname = input("Please introduce fighter A name")


fighter_b = input("Please introduce fighter B id")
division = int(input("Please introduce division (1-Flyweight, 4-Lightweight, 8-Heavyweight)"))
fight_date = input(f"Please introduce the fight date in format like 'Jan 1, 2000'")
fight_datetime = datetime.strptime(fight_date.strip(), "%b %d, %Y").date()

predict_hypothetical_probabilities(fighter_a, fighter_b, division, fight_datetime)

Please introduce fighter A id e1248941344b3288
Please introduce fighter B id f166e93d04a8c274
Please introduce division (1-Flyweight, 4-Lightweight, 8-Heavyweight) 3
Please introduce the fight date in format like 'Jan 1, 2000' Jan 1, 2026


C:\Users\Rober\miniconda3\envs\ufc\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


Gaussian: 0.40802442364532965
XG Boost: 0.4487445652484894


C:\Users\Rober\miniconda3\envs\ufc\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


Gaussian: 0.45690687564524524
XG Boost: 0.4747301936149597
Probabilities given for Alexander Volkanovski to beat Diego Lopes in LW: 52.0%


**TO DO: TRY OTHER MODELS**

In [ ]:
# !pip install torch

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error


class NN(nn.Module):
    def __init__(self, input_dim, drop_rate=0.5):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.dropout1 = nn.Dropout(drop_rate)
        self.fc2 = nn.Linear(256, 128)
        self.dropout2 = nn.Dropout(drop_rate)
        self.fc3 = nn.Linear(128, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout1(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)
        return x

In [ ]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression()
reg.fit(X_train, y_train)
reg.score(X_val, y_val)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

reg = RandomForestRegressor()
reg.fit(X_train, y_train)
reg.score(X_val, y_val)